In [ ]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

df = pd.read_csv("fruit_vegetable_pricing_dataset.csv")

X = df.drop(columns=["Optimal Selling Price"])
y = df["Optimal Selling Price"]

categorical_features = ["Item Name"]
preprocessor = ColumnTransformer(
    transformers=[("cat", OneHotEncoder(handle_unknown='ignore'), categorical_features)],
    remainder='passthrough'
)

model = make_pipeline(preprocessor, LinearRegression())

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train)

with open("pricing_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("Model saved as pricing_model.pkl")


Model saved as pricing_model.pkl


In [ ]:
import pickle
import pandas as pd

# Load the trained model
with open("pricing_model.pkl", "rb") as f:
    model = pickle.load(f)

# Print the pipeline steps to confirm model step name
print(model.named_steps)  # Debugging step

# User input
item_name = input("Enter Item Name: ").strip()
quantity = int(input("Enter Quantity: "))

# Create input DataFrame (initialize missing numeric columns with 0)
input_data = pd.DataFrame([[item_name, quantity, 0, 0, 0, 0, 0, 0, 0]],
                          columns=["Item Name", "Quantity", "Purchase Cost", "Market Demand", "Seasonality",
                                   "Competitor Price", "Previous Selling Price", "Sales Velocity", "Profit Margin"])

# Transform input using the preprocessor
input_transformed = model.named_steps["columntransformer"].transform(input_data)

# Identify the correct model step name (e.g., "ridge" or another name)
model_step_name = list(model.named_steps.keys())[-1]  # The last step is usually the ML model

# Predict using the trained model
predicted_price = model.named_steps[model_step_name].predict(input_transformed)[0]

print(f"Predicted Optimal Selling Price: ${predicted_price:.2f}")


{'columntransformer': ColumnTransformer(remainder='passthrough',
                  transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Item Name'])]), 'linearregression': LinearRegression()}
Enter Item Name: apple
Enter Quantity: 74
Predicted Optimal Selling Price: $-7.19


In [ ]:
print(model.named_steps)


{'columntransformer': ColumnTransformer(remainder='passthrough',
                  transformers=[('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['Item Name'])]), 'linearregression': LinearRegression()}
